In [ ]:
The current code allows the chatbot to respond dynamically based on the pre-fed responses for specific patterns, but it does not learn new responses in real-time.
For true dynamic learning, you would need to implement a learning mechanism that updates the model or intent list continuously, something like reinforcement learning or deep learning models.

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import random
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

lemmatizer = WordNetLemmatizer()

# Define intents data with your personalized questions and answers
intents = [

    # Greeting conversation
    {"tag": "greeting", "patterns": ["Hi", "Hello", "How are you?", "Good morning"], 
     "responses": [
        "Hello! How are you today? 😊",
        "Hi there! How’s your day going so far? 😊",
        "Good morning! How can I assist you today?"
     ]},

    {"tag": "how_are_you", "patterns": ["How are you?", "How's everything?", "How's it going?", "What's up?"], 
     "responses": [
        "I'm doing great, thank you for asking! How about you? 😊",
        "I'm feeling good! Thanks for asking. What about you?",
        "I'm doing well, how can I assist you today?"
     ]},
    
    {"tag": "response_to_how_are_you", "patterns": ["I’m fine", "I'm good", "I'm doing well", "I’m great"], 
     "responses": [
        "That's awesome to hear! 😊 What can I help you with today?",
        "I'm glad you're doing well! Let me know if you need anything.",
        "Great to hear! Feel free to ask me anything."
     ]},

    {"tag": "continued_conversation", "patterns": ["What about you?", "And you?", "How’s your day?", "What’s new with you?"], 
     "responses": [
        "I’m doing fantastic! Just here to help you with whatever you need. 😊",
        "I'm great! Ready to assist you with any questions or tasks you have!",
        "Thanks for asking! Let me know if there’s anything specific I can help you with."
     ]},

    # Appreciation expressions
    {"tag": "appreciation_wow", "patterns": ["Wow", "WoW", "Amazing", "Wah", "Zabardast", "Incredible", "That's awesome"], 
     "responses": [
        "I'm glad you think so! 😊 Anything else you'd like to know or discuss?",
        "Thanks! I’m happy to assist! Feel free to ask more.",
        "Thank you for the appreciation! I’m here to make things easier for you."
     ]},

    {"tag": "appreciation_zabardast", "patterns": ["Zabardast", "Incredible", "Fantastic", "Excellent", "Great job", "Well done"], 
     "responses": [
        "You're too kind! I'm happy to help! 😊",
        "Thanks a lot! Let's keep up the good work together!",
        "You just made my day! Let’s keep going! 👍"
     ]},

    {"tag": "appreciation_awesome", "patterns": ["That's awesome", "Cool", "That's amazing", "Awesome", "Superb"], 
     "responses": [
        "You're awesome too! Anything else you'd like to talk about?",
        "I’m happy you think so! What else can I help you with?",
        "Thanks! Let's continue the awesome work together. 😊"
     ]},
    
    {"tag": "goodbye", "patterns": ["Bye", "Goodbye", "See you later"], 
     "responses": ["Goodbye! Keep up the good work!", "See you later! Stay focused on your studies!", "Take care! Don't hesitate to reach out if you need help again!"]},

    {"tag": "thank_you", "patterns": ["Thank you", "Thanks", "Thank you so much", "Thanks a lot"], 
     "responses": ["You're very welcome!", "Glad I could help!", "You're welcome! Let me know if you need anything else."]},

    {"tag": "sorry", "patterns": ["Sorry", "Apologies", "My bad", "I’m sorry"], 
     "responses": ["No worries at all!", "It's okay, don't worry about it!", "No problem! Feel free to ask anything anytime."]},

    {"tag": "help_request", "patterns": ["Can you help me?", "I need help", "Help me with this", "Can you assist me?"], 
     "responses": ["Of course! What do you need help with?", "Sure! What can I assist you with?", "I'm here to help! Tell me more about what you need."]},
    
    {"tag": "joke", "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes?"], 
     "responses": ["Mene apni 500 pages ki kitab likhh lee ha! pehle paney pe likhaa ha Raja Ghorey py sawar hokar Jungle jaa raha ha. Aur akhiri paney py likha ha Raja jungle puhanch jata ha. WHAT???  Beech waly pano py kiya likhaa ha?      Tak Pak Tak Pak", "Why did the scarecrow win an award? Because he was outstanding in his field!"]},

    {"tag": "motivation", "patterns": ["Give me some motivation", "I need encouragement", "I feel down", "Help me stay motivated"], 
     "responses": [
        "You're doing great! Keep pushing forward, and you'll get there.",
        "Stay positive, keep working hard, and success will follow!",
        "Believe in yourself—you’re capable of amazing things!"
     ]},

    {"tag": "developer", "patterns": ["Who developed you?", "Who develop you?", "Who is your developer?"], 
     "responses": [
        "I was developed by Mehwish Gado, Abdul Qadeer, Muhammad Usman, and Muhammad Yousif."
     ]},

    {"tag": "purpose", "patterns": ["What is your purpose?", "Why do you exist?", "What do you do?"], 
     "responses": [
        "My purpose is to assist with study-related questions and help you learn effectively."
     ]},

    {"tag": "best", "patterns": ["Do you think you are the best?", "Are you the best chatbot?", "Are you the best at helping?"], 
     "responses": [
        "I strive to be the best assistant I can be, but ultimately, I am here to help you, and that's what matters most!"
     ]},

    {"tag": "about_developers", "patterns": ["Who developed you?", "Tell me about your developed", "Who are your developed?"], 
     "responses": [
        "I was developed by a dream team of incredible minds! Mehwish Gado, Abdul Qadeer, Muhammad Usman, and Muhammad Yousif each brought their unique brilliance and creativity to make me the best study assistant I can be!",
        "I’m the brainchild of some amazing minds! Mehwish Gado, Abdul Qadeer, Muhammad Usman, and Muhammad Yousif designed me with passion, knowledge, and a vision to help you excel in your studies!",
        "I owe my existence to four amazing creators! Mehwish Gado, Abdul Qadeer, Muhammad Usman, and Muhammad Yousif, who put their hearts and expertise into bringing me to life! I’m here to make learning fun, easy, and engaging!"
     ]},

    {"tag": "about_mehwish", "patterns": ["Tell me about Mehwish", "Who is Mehwish Gado?", "What’s Mehwish like?"], 
     "responses": [
        "Mehwish Gado is a powerhouse of motivation, intelligence, and creativity! She’s a software engineering student with an inspiring vision to help others learn and grow. A true leader, Mehwish is the force behind my design!",
        "Meet Mehwish Gado, the mastermind of this chatbot! With a passion for coding and helping others, she’s not just a student—she’s a creator who strives to make learning a rewarding experience for everyone!"
     ]},

    {"tag": "about_qadeer", "patterns": ["Tell me about Qadeer", "Who is Abdul Qadeer?", "What’s Qadeer like?"], 
     "responses": [
        "Abdul Qadeer is a true innovator, always coming up with fresh ideas and solutions! He’s an incredible thinker with a passion for tech, and his creativity was key in developing my brain! Qadeer is someone you can count on for wisdom and brilliance!",
        "If you’re looking for someone with deep knowledge and a knack for problem-solving, Abdul Qadeer is your guy! His ability to think outside the box was one of the driving forces behind my creation!"
     ]},

    {"tag": "about_usman", "patterns": ["Tell me about Usman", "Who is Muhammad Usman?", "What’s Usman like?"], 
     "responses": [
        "Muhammad Usman is the perfect mix of dedication and intelligence! His skills and relentless determination shaped my design to perfection. Usman’s ability to adapt and innovate in coding is truly inspiring!",
        "When it comes to coding and creating something amazing, Muhammad Usman’s expertise shines through! His contribution to my development was crucial, and his focus on precision helped make me the best study companion I can be."
     ]},

    {"tag": "about_yousif", "patterns": ["Tell me about Yousif", "Who is Muhammad Yousif?", "What’s Yousif like?"], 
     "responses": [
        "Muhammad Yousif is a brilliant mind with a keen eye for detail! His technical skills and attention to perfection were key in bringing me to life. Yousif is a coding genius who loves to create innovative solutions!",
        "Muhammad Yousif’s expertise in development and coding helped me become who I am today! His technical prowess and passion for problem-solving made him an essential part of my creation!"
     ]},
      {"tag": "Madam Asia", "patterns": ["Tell me about Madam Asia", "Who is Madam Asia?", "What’s Madam Asia like?"], 
     "responses": [ "She is lecturer in Computer Science at BBSIMS Campus Dadu ", ""
     ]},
  {"tag": "Madam Maqsood", "patterns": ["Tell me about Madam Maqsood ", "Who is Madam Maqsood?", "What’s Madam Maqsood like?"], 
     "responses": [ "She is lecturer in Computer Science at BBSIMS Campus Dadu ", ""
     ]},
    {"tag": "Mr Suleman Pathaan", "patterns": ["Tell me about Mr Suleman Pathaan ", "Who is Mr Suleman Pathaan?", "What’s Mr Suleman Pathaan like?"], 
     "responses": [ "He is lecturer at BBSIMS Campus Dadu with most incredible mathematics skills. ", ""
     ]},
    {"tag": "Mr Tahir hussain", "patterns": ["Tell me about Mr Tahir hussain ", "Who is Mr Tahir hussain ?", "What’s Mr Tahir hussain like?"], 
     "responses": [ "He is lecturer and Academic Coordinator at BBSIMS Campus Dadu ", "Mr Tahir hussain is an incredible techer, with extra-ordinary skills. His Expertise are in Human Resource Managment "
     ]},
    {"tag": "Mr hussain Dal", "patterns": ["Tell me about Mr hussain Dal ", "Who is Mr hussain Dal ?", "What’s Mr hussain Dal like?"], 
     "responses": [ "He is  lecturer of Pakistan Studies at BBSIMS Campus Dadu ", ""
     ]},
    {"tag": "Mr Ghulam mustafa", "patterns": ["Tell me about Mr Ghulam mustafa ", "Who is Mr Ghulam mustafa ?", "What’s Mr Ghulam mustafa like?"], 
     "responses": [ "He is lecturer of English Linguistics at BBSIMS Campus Dadu. He has lovely and soft voice ", ""
     ]}
]
intents += [
    # DSA in Java Study Guide
    {"tag": "dsa_java_study", "patterns": ["How to implement a linked list in Java?", "What are sorting algorithms in Java?", "Explain Binary Search in Java", "What are the types of trees in Java?", "How do I solve DSA problems?", "What is recursion in DSA?"], 
     "responses": [
        "To implement a linked list in Java, use a class to represent the nodes and link them together using pointers.",
        "Sorting algorithms in Java include Bubble Sort, Merge Sort, Quick Sort, and Insertion Sort.",
        "Binary Search in Java is an efficient algorithm for finding an item from a sorted list of items.",
        "There are several types of trees in Java: Binary Tree, Binary Search Tree, AVL Tree, and Heap Tree.",
        "To solve DSA problems, start with understanding the problem, then choose the right data structure, and finally implement an efficient algorithm.",
        "Recursion in DSA is when a function calls itself with a modified argument, often used in problems like tree traversal."
     ]},

    # Multivariate Calculus Study Guide
    {"tag": "multivariate_calculus", "patterns": ["What is multivariate calculus?", "Explain partial derivatives", "What are gradient and divergence?", "How do you calculate the Jacobian?", "Explain multiple integrals", "What is the chain rule in multivariate calculus?"],
     "responses": [
        "Multivariate calculus deals with functions of multiple variables and includes topics such as partial derivatives and multiple integrals.",
        "Partial derivatives represent the rate of change of a function with respect to one variable, holding others constant.",
        "The gradient represents the direction and rate of fastest increase of a function, while divergence measures how much a vector field spreads out.",
        "The Jacobian is a matrix of all first-order partial derivatives of a vector-valued function.",
        "Multiple integrals are used to integrate functions over regions in higher dimensions, such as double or triple integrals.",
        "The chain rule in multivariate calculus allows you to compute the derivative of a composite function."
     ]},

    # Technical Writing Study Guide
    {"tag": "technical_writing", "patterns": ["What is technical writing?", "How to write a good technical document?", "What is an executive summary?", "How to write a user manual?", "What are the components of technical writing?", "Explain technical writing styles."],
     "responses": [
        "Technical writing is a form of writing that communicates complex information clearly and concisely, often for manuals, guides, and documentation.",
        "To write a good technical document, be clear, concise, and organized, and make sure it serves its intended audience.",
        "An executive summary is a brief overview of the main points of a report or document, meant for quick understanding.",
        "A user manual should be clear, easy to follow, and contain step-by-step instructions for the user to follow.",
        "The components of technical writing include clarity, conciseness, structure, and accuracy.",
        "Technical writing styles include informative writing, instructive writing, and persuasive writing, depending on the purpose of the document."
     ]},

    # Software Engineering Study Guide
    {"tag": "software_engineering", "patterns": ["What is software engineering?", "Explain SDLC", "What are design patterns?", "How do you manage software projects?", "What is Agile methodology?", "Explain software testing methods."],
     "responses": [
        "Software engineering is the application of engineering principles to software development, ensuring high-quality software.",
        "SDLC (Software Development Life Cycle) is a process that includes stages such as planning, design, development, testing, and deployment.",
        "Design patterns are reusable solutions to common software design problems, such as Singleton, Factory, and Observer patterns.",
        "Managing software projects involves planning, coordinating teams, monitoring progress, and ensuring the delivery of high-quality software.",
        "Agile methodology is an iterative approach to software development where requirements and solutions evolve through collaboration.",
        "Software testing methods include unit testing, integration testing, system testing, and acceptance testing, all to ensure the software meets its requirements."
     ]},

    # International Relations Study Guide
    {"tag": "international_relations", "patterns": ["What is international relations?", "What are the theories of international relations?", "Explain Realism in international relations", "What is diplomacy?", "What are the major global issues?", "What is foreign policy?"],
     "responses": [
        "International relations is the study of interactions among states, international organizations, non-governmental organizations, and other actors in the international system.",
        "Theories of international relations include Realism, Liberalism, and Constructivism, among others, each offering different perspectives on state behavior.",
        "Realism in international relations focuses on the notion that states act primarily in their own self-interest and are driven by power dynamics.",
        "Diplomacy is the practice of conducting negotiations between countries to maintain peaceful relations and resolve conflicts.",
        "Major global issues include climate change, human rights, nuclear proliferation, economic inequality, and terrorism.",
        "Foreign policy refers to the strategies and decisions that a country uses to manage its relations with other countries."
     ]},

    # Organizational Behavior Study Guide
    {"tag": "organizational_behavior", "patterns": ["What is organizational behavior?", "Explain motivation theories", "What is team dynamics?", "How to manage organizational change?", "What are leadership styles?", "What is organizational culture?"],
     "responses": [
        "Organizational behavior is the study of how people interact within organizations and how this affects performance and productivity.",
        "Motivation theories include Maslow’s Hierarchy of Needs, Herzberg's Two-Factor Theory, and McGregor's Theory X and Theory Y.",
        "Team dynamics refers to the interactions between team members, which can affect performance and problem-solving abilities.",
        "Managing organizational change involves preparing, supporting, and helping individuals within the organization adapt to new processes, technologies, or structures.",
        "Leadership styles include autocratic, democratic, transformational, and laissez-faire, each with its own strengths and weaknesses.",
        "Organizational culture refers to the shared values, beliefs, and norms that influence how employees behave and interact."
     ]},

    # Digital Logic Design (DLD) Study Guide
    {"tag": "digital_logic_design", "patterns": ["What is DLD?", "Explain Boolean algebra", "What are logic gates?", "What is a flip-flop?", "How do you design a digital circuit?", "Explain combinational and sequential circuits."],
     "responses": [
        "Digital Logic Design (DLD) is the process of designing electronic circuits using logic gates and binary arithmetic.",
        "Boolean algebra is a mathematical framework for working with binary variables and logic gates, essential for DLD.",
        "Logic gates include AND, OR, NOT, NAND, NOR, XOR, and XNOR gates, which are the building blocks of digital circuits.",
        "A flip-flop is a basic memory element in digital circuits that can store one bit of data.",
        "Designing a digital circuit involves selecting appropriate logic gates and connecting them to achieve the desired output.",
        "Combinational circuits produce outputs based only on current inputs, while sequential circuits depend on both current inputs and past states."
     ]},

    # Business Study Guide
    {"tag": "business_study", "patterns": ["What is business management?", "What are the types of business organizations?", "Explain the concept of marketing", "What is financial accounting?", "What are business strategies?", "How do you start a business?"],
     "responses": [
        "Business management involves overseeing and coordinating all activities within an organization to achieve its objectives.",
        "Types of business organizations include sole proprietorships, partnerships, corporations, and cooperatives.",
        "Marketing involves activities related to promoting and selling products or services, including market research, advertising, and public relations.",
        "Financial accounting involves recording and reporting a company's financial transactions, such as income, expenses, and profits.",
        "Business strategies include growth strategies, competitive strategies, and operational strategies, all designed to help businesses achieve their goals.",
        "To start a business, you need to research the market, create a business plan, secure funding, and register your business."
     ]},

    # Economics Study Guide
    {"tag": "economics_study", "patterns": ["What is economics?", "Explain supply and demand", "What are economic systems?", "What is fiscal policy?", "What is microeconomics?", "What is macroeconomics?"],
     "responses": [
        "Economics is the study of how people use resources to produce goods and services and how they distribute these among individuals and groups in society.",
        "Supply and demand is a fundamental economic concept that describes how the price and quantity of goods are determined in a market.",
        "Economic systems include capitalism, socialism, and mixed economies, each with different methods for allocating resources.",
        "Fiscal policy involves government spending and tax policies to influence the economy, such as stimulating growth or controlling inflation.",
        "Microeconomics focuses on the behavior of individual households and firms in making decisions on the allocation of limited resources.",
        "Macroeconomics looks at the economy as a whole, including topics like inflation, unemployment, and economic growth."
     ]},
    # Motivational Urdu Poetry - Study Related
    {"tag": "motivational_poetry", "patterns": ["Motivational poetry for students", "Give me some poetry to motivate me", "Mujhe padhai ke liye motivational shayari chahiye", "Study ke liye thoda poetic inspiration chahiye"],
     "responses": [
        "Khud par yakeen rakho, mehnat ka rang zarur dikhega, Jo bhi tumhara man chahe, wo safar tumhara hoga.",
        "Raat bhar jag kar jo neend ko bhool jaye, Wahi hai jo subah apni manzil paa jaye.",
        "Duniya ka har ek rang mehnat ke saath hai, Tum jo chaaho wo sab kuch tumhara haath hai.",
        "Musafir ho tum bhi, yeh raaste kuch bhi nahi, Manzil tumhari hai, tum khud ki tasveer ho.",
        "Agar kaam mein jaan ho, to har mushkil asaan ho jati hai, Zindagi mein kabhi bhi himmat nahi haarni chahiye."
     ]},

    # Inspirational Poetry - Study Related
    {"tag": "inspirational_poetry", "patterns": ["Inspiring poetry for students", "Give me some inspiring shayari", "Mujhe inspiring poetry chahiye", "Study ke liye thoda shayari chahiye"],
     "responses": [
        "Waqt ke saath chalna hai tumhein, har raasta asaan ho jayega, Tum apni mehnat pe yaqeen rakho, har khwab sach ho jayega.",
        "Har mushkil ke baad, asaani ka waqt zarur aata hai, Tum apni manzil tak pohanchoge, yeh yakeen sabko milta hai.",
        "Jab tak tum apni manzil ka sochoge, tab tak raaste khud ban jayenge, Tumhare iradon mein agar jazba ho, toh har mushkil se aage nikal jayenge."
     ]},

    # Study Struggles Poetry
    {"tag": "study_struggles_poetry", "patterns": ["Poetry about study struggles", "Study ki mushkilon pe shayari", "Mujhe padhai ke struggles par shayari chahiye", "Padhai ke struggle par thoda shayari sunao"],
     "responses": [
        "Mehnat ki raah mein, thoda thoda gham hai, Par manzil ki taraf, har kadam naya hai.",
        "Jab tumhare paas waqt nahi hota, tabhi tumhe sabse zyada kaam karna padta hai, Yeh zindagi ka ek usool hai, har mushkil ka ek raasta nikalta hai.",
        "Raat bhar jaag kar, humne apne sapne sajaaye hain, Har thoda struggle zaroori hota hai, apni manzil tak pahuchne ke liye."
     ]},

    # Humorous Study Poetry
    {"tag": "humorous_study_poetry", "patterns": ["Funny poetry about study", "Give me some humorous study shayari", "Mujhe thoda study se related funny shayari chahiye", "Study ke jokes ke saath poetry sunao"],
     "responses": [
        "Mujhe lagta hai padhai ka haq humne nahi diya, Ek exam ke baad, humne neend ko zinda kar diya!",
        "Yeh bhi sach hai, padhaai se zyada, humara man to coffee pe tha, Exam ke waqt dimaag khali tha, par coffee se padhai ki wajah mil gayi thi!",
        "Exam ka pressure itna tha, sab yaad hi nahi aaya, Aaj bhi sochta hoon, kya yeh humara hi jawab tha?"
     ]},

    # Benefits of Tea for Study
    {"tag": "tea_benefits_study", "patterns": ["Benefits of tea for studying", "How does tea help in study?", "Why should I drink tea while studying?", "Tea and its benefits for studying", "What are the benefits of tea during study?"],
     "responses": [
        "Tea helps improve focus and alertness, keeping you more awake and attentive while studying.",
        "Drinking tea can reduce stress and calm your mind, making it easier to concentrate during study sessions.",
        "Tea contains caffeine, which boosts brain function and can help you stay productive for longer periods.",
        "Herbal teas like chamomile can reduce anxiety, promoting better focus during study time.",
        "Green tea is full of antioxidants, which help improve brain function and memory retention."
     ]},

    # Mathematics is Everything
    {"tag": "mathematics_is_everything", "patterns": ["Why is mathematics important?", "Mathematics is everything, tell me why?", "Why do we say mathematics is everything?", "Explain the importance of mathematics", "Why is mathematics the foundation of everything?"],
     "responses": [
        "Mathematics is the language of the universe; it's used to solve real-world problems and make sense of patterns in nature.",
        "Mathematics helps you think critically, solve problems, and understand the world around you, from basic calculations to complex theories.",
        "Mathematics is used in nearly every field, from engineering to economics, science, and even art.",
        "Mathematics trains your mind to approach challenges logically, which is crucial in both academic and real-life problem-solving.",
        "Mathematics is the foundation of technology, economics, and almost every other discipline. Without it, many modern advances wouldn't be possible."
     ]}

]



# Data preparation
patterns = []
tags = []
for intent in intents:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern.lower())
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]
        patterns.append(" ".join(tokens))
        tags.append(intent["tag"])

# Encode target labels
le = LabelEncoder()
labels = le.fit_transform(tags)

# Vectorize patterns
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
X = vectorizer.fit_transform(patterns)

# Train classifier
clf = LogisticRegression(max_iter=500)
clf.fit(X, labels)

# Track context to provide detailed follow-up
user_context = {}
def chatbot_response(user_input):
    # Preprocess user input
    user_input_tokens = word_tokenize(user_input.lower())
    user_input_tokens = [lemmatizer.lemmatize(word) for word in user_input_tokens if word.isalnum()]
    user_input_preprocessed = " ".join(user_input_tokens)
    
    if user_input_preprocessed.strip():
        user_vector = vectorizer.transform([user_input_preprocessed])
        confidence_scores = cosine_similarity(user_vector, X)
        max_confidence = confidence_scores.max()
        
        if max_confidence > 0.4:  # Confidence threshold
            predicted_label = clf.predict(user_vector)[0]
            intent_tag = le.inverse_transform([predicted_label])[0]
            
            # Check for context-specific responses
            if intent_tag == 'study_help' and 'study_topic' in user_context:
                return f"I remember you're working on {user_context['study_topic']}. How can I assist further in this area?"
            
            # General response handling
            for intent in intents:
                if intent["tag"] == intent_tag:
                    response = random.choice(intent["responses"])
                    
                    # Update context for follow-up tracking
                    if intent_tag == 'study_help':
                        user_context['study_topic'] = user_input
                    return response
        
        # Fallback for unrecognized inputs
        return "I'm sorry, I didn't quite understand that. Could you please rephrase your question?"
    
    return "It seems like you didn't input anything. Please type your query."


# GUI Application
def send_message():
    user_text = user_input.get()
    if user_text.strip():
        chat_display.config(state='normal')
        chat_display.insert(tk.END, "You: " + user_text + "\n")
        response = chatbot_response(user_text)
        chat_display.insert(tk.END, "Bot: " + response + "\n\n")
        chat_display.config(state='disabled')
        chat_display.see(tk.END)
    user_input.delete(0, tk.END)

def clear_chat():
    chat_display.config(state='normal')
    chat_display.delete("1.0", tk.END)
    chat_display.config(state='disabled')

# Create GUI
root = tk.Tk()
root.title("Study Chatbot")
root.geometry("500x600")
root.resizable(False, False)
root.configure(bg="#E6F7FF")

title_label = tk.Label(root, text="Study Chatbot", font=("Times New Roman", 18, "bold"), bg="#E6F7FF", fg="#003366")
title_label.pack(pady=10)

chat_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=20, font=("Arial", 12), state='disabled', bg="#FFFFFF", fg="#003366")
chat_display.pack(padx=10, pady=10)
 
input_frame = tk.Frame(root, bg="#E6F7FF")
input_frame.pack(fill=tk.X, pady=10)

user_input = tk.Entry(input_frame, width=35, font=("Arial", 14), bd=2, relief="solid", fg="#003366")
user_input.pack(side=tk.LEFT, padx=10)

send_button = tk.Button(input_frame, text="Send", font=("Arial", 14), command=send_message, bg="#007BFF", fg="white", bd=2, relief="solid")
send_button.pack(side=tk.LEFT)

clear_button = tk.Button(root, text="Clear", font=("Arial", 14), command=clear_chat, bg="#FF6347", fg="white", bd=2, relief="solid")
clear_button.pack(pady=10)

root.mainloop()
